# Query Test with Mixed Visa Guide Doc + FAQ Doc

E9 Visa Guide 문서와 E9 Visa FAQ 문서를 믹스하여 사용자의 질문에 원하는 결과가 나오는지 테스트

테스트환경
* LLM: gpt-4-1106-preview
* Embedding: text-embedding-ada-002
* VectorStore: default vector store
        

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

#### LLM 세팅

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4-1106-preview", temperature=0)

#### Load Files

1. E-9 Visa Guide_한국어.pdf
2. E-9 FAQ_한국어.pdf

In [16]:
"""Load from pdf file"""

import json
from llama_index.readers.file import PyMuPDFReader
from llama_index.core import SimpleDirectoryReader

root_path = "../../../Documents"

visa_pdf_file_path = f"{root_path}/E-9 Visa Guide_한국어.pdf"
visa_docs = PyMuPDFReader().load_data(file_path=visa_pdf_file_path, metadata=True)


faq_json_file_path = f"{root_path}/E9_Faq.json"
with open(faq_json_file_path) as f:
    faq_json_list = json.load(f)


#### Mix Guide Document and FAQ Document

In [37]:
from llama_index.core.schema import TextNode

visa_nodes = []
faq_nodes = []

for faq_idx, faq_doc in enumerate(faq_json_list):
    node = TextNode(
        text=faq_doc['question'], 
        metadata={"answer": faq_doc['answer'], "file_name": "E9 Visa FAQ"},
        excluded_llm_metadata_keys=["file_name"]
    )
    faq_nodes.append(node)

for idx, visa_doc in enumerate(visa_docs):
    if idx < 2:
        continue
    node = TextNode(
        text=visa_doc.text, 
        metadata={**visa_doc.metadata, "file_name": "E9 Visa Guide"},
        excluded_llm_metadata_keys=["page_label", "file_path", "total_pages", "file_type", "file_size", "creation_date", "last_modified_date", "source", "file_name"],
    )
    visa_nodes.append(node)


all_nodes = visa_nodes + faq_nodes

#### Total count of chunks is 57

In [38]:
print(f"Count of all nodes: {len(all_nodes)}")

Count of all nodes: 57


#### Load VectoreStore with All Nodes 

In [39]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(all_nodes, show_progress=True)

Generating embeddings:   0%|          | 0/57 [00:00<?, ?it/s]

In [44]:
def pretty_print_sources(source_nodes):
    for source_node in source_nodes:
        print(f"Score: {source_node.score:.3f}")
        print(f"Metadata: {source_node.metadata['file_name']}")
        print(f"Text: {source_node.get_content()[:100]}")
        print(f"-------------------------------------------------------------------------------\n\n")

#### 유저 질문을 통한 청크테스트

In [40]:
retriever = index.as_retriever(similarity_top_k=20)

아래 E9 Visa Guide 를 위한 질문(Q: 사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?)에 대해선 질문에 대한 적절한 청크가 2번째로 뽑힘

In [49]:
query = "사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?" # E9 Visa Guide 를 위한 질문

source_nodes = retriever.retrieve(query)
pretty_print_sources(source_nodes)

Score: 0.855
Metadata: E9 Visa FAQ
Text: 고용허가제로 한국에 갈건데, 한국 가서 교육 받을 수 있는지, 아니면 미리 자격증 있어야 신청할 수 있는지 알고 싶어요. 예를 들어서 농업 쪽에 일하려고 하는데 농업에 대한 지식이
-------------------------------------------------------------------------------


Score: 0.848
Metadata: E9 Visa Guide
Text: 고용허가제해당자재입국특례자에대한사증신청및발급방법
●
사증발급방법
○
출입국ㆍ외국인청(사무소ㆍ출장소)장이발급한사증발급인정서에의해
사증발급
○
사증신청은송출기관에의한단체신청만가능하고외
-------------------------------------------------------------------------------


Score: 0.847
Metadata: E9 Visa Guide
Text: 고용허가제해당자는2012.8.1부터범죄경력증명서및
건강상태확인서제출
●
범죄경력확인관련
○
확인서류: 국적국의권한있는기관이발급한범죄경력증명서
■
범죄경력증명서의요건
●
자국내의모든
-------------------------------------------------------------------------------


Score: 0.843
Metadata: E9 Visa Guide
Text: 고용허가제해당자는사증발급인정서를받아야만E-9 사증을
발급받을수있음
●
비전문취업(E-9) 사증은출입국ㆍ외국인청(사무소ㆍ출장소)장이발급한
사증발급인정서에의해서만사증을발급할수있음

-------------------------------------------------------------------------------


Score: 0.842
Metadata: E9 Visa FAQ
Text: 중학교 졸업증만 있는데 한국에서 일할 수 있어요?
-------------------------------

#### Cohere Reranker Test

Cohere Reranker를 통해 순위 재조정
E9 Visa Guide 를 위한 질문(Q: 사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?)에 적절한 청크가 뽑힘

In [58]:
from llama_index.postprocessor.cohere_rerank import CohereRerank


query = "사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?" # E9 Visa Guide 를 위한 질문
source_nodes = retriever.retrieve(query)

reranker = CohereRerank(
    top_n=3, model="rerank-multilingual-v2.0"
)

filtered_nodes = reranker.postprocess_nodes(source_nodes, query_str=query)

pretty_print_sources(filtered_nodes)

Score: 0.935
Metadata: E9 Visa Guide
Text: 고용허가제비전문취업(E-9) 자격사증발급인정서발급절차
●
접수/신청방법
○
비전문취업자를초청하고자하는사업장의장이사업장주소지를관할하는
출입국ㆍ외국인청(사무소ㆍ출장소)장에게직접방문또는
-------------------------------------------------------------------------------


Score: 0.889
Metadata: E9 Visa Guide
Text: 고용허가제해당자는2012.8.1부터범죄경력증명서및
건강상태확인서제출
●
범죄경력확인관련
○
확인서류: 국적국의권한있는기관이발급한범죄경력증명서
■
범죄경력증명서의요건
●
자국내의모든
-------------------------------------------------------------------------------


Score: 0.283
Metadata: E9 Visa Guide
Text: 고용허가제해당자의재입국허가
●
재입국허가면제제도시행(’10.12.1.자개정시행규칙)
○
등록을필한외국인이출국한날부터1년이내에재입국하려는경우재입국허가
면제
○
체류기간이1년보다적게남
-------------------------------------------------------------------------------




In [51]:
query = "중학교 졸업증만 있는데 한국에서 일할 수 있어요?" # E9 Visa FAQ 를 위한 질문, FAQ에 있는 질문 내용이랑 같은 질문입니다.

source_nodes = retriever.retrieve(query)
pretty_print_sources(source_nodes)

Score: 0.893
Metadata: E9 Visa FAQ
Text: 중학교 졸업증만 있는데 한국에서 일할 수 있어요?
-------------------------------------------------------------------------------


Score: 0.871
Metadata: E9 Visa FAQ
Text: 학력 프로그램 수료증이 고등학교 졸업장이 아닌 경우 E9 로 한국 갈 수 있나요?
-------------------------------------------------------------------------------


Score: 0.851
Metadata: E9 Visa FAQ
Text: 고용허가제로 한국에 갈건데, 한국 가서 교육 받을 수 있는지, 아니면 미리 자격증 있어야 신청할 수 있는지 알고 싶어요. 예를 들어서 농업 쪽에 일하려고 하는데 농업에 대한 지식이
-------------------------------------------------------------------------------


Score: 0.845
Metadata: E9 Visa FAQ
Text: 시력이 안 좋아도 한국에 갈 수 있을까요?
-------------------------------------------------------------------------------


Score: 0.835
Metadata: E9 Visa FAQ
Text: 저 여자인데 한국에서 식당에서 일 할 수 있나요?
-------------------------------------------------------------------------------


Score: 0.830
Metadata: E9 Visa FAQ
Text: 올해  한국에 농업 일로 접수되는 사람 많아요?
-------------------------------------------------------------------------------


Score: 0

In [52]:
query = "저는 베트남에서 중학교에 간후 졸업했는데, 이럴때 고용허가제를 통해 직업을 얻을 수 있나요?" #위의 질문을 변형함

source_nodes = retriever.retrieve(query)
pretty_print_sources(source_nodes)

Score: 0.861
Metadata: E9 Visa FAQ
Text: 중학교 졸업증만 있는데 한국에서 일할 수 있어요?
-------------------------------------------------------------------------------


Score: 0.852
Metadata: E9 Visa FAQ
Text: 학력 프로그램 수료증이 고등학교 졸업장이 아닌 경우 E9 로 한국 갈 수 있나요?
-------------------------------------------------------------------------------


Score: 0.841
Metadata: E9 Visa FAQ
Text: 고용허가제로 한국에 갈건데, 한국 가서 교육 받을 수 있는지, 아니면 미리 자격증 있어야 신청할 수 있는지 알고 싶어요. 예를 들어서 농업 쪽에 일하려고 하는데 농업에 대한 지식이
-------------------------------------------------------------------------------


Score: 0.838
Metadata: E9 Visa FAQ
Text: 외국인 E9 비자 고용보험 가입해야 하나요? 
-------------------------------------------------------------------------------


Score: 0.833
Metadata: E9 Visa FAQ
Text: 시력이 안 좋아도 한국에 갈 수 있을까요?
-------------------------------------------------------------------------------


Score: 0.830
Metadata: E9 Visa FAQ
Text: E9으로 농업 분야에서 일하고 싶은데 100프로 한국에 가서 일하는 걸 보장할 수 있나요?
--------------------------------------------------------------------

#### Load Cohere Reranker and set Filter with Query Engine

유사성검색을 통해 상위 20개의 청크를 뽑고, `Cohere Reranker`를 통해서 마지막 3개의 청크를 뽑는다.

In [59]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

reranker = CohereRerank(
    top_n=3, model="rerank-multilingual-v2.0"
)

query_engine = index.as_query_engine(
    streaming=True, 
    similarity_top_k=20,
    node_postprocessors=[reranker],
)

In [60]:
query = "중학교 졸업증만 있는데 한국에서 일할 수 있어요?" # E9 Visa FAQ 를 위한 질문, FAQ에 있는 질문 내용이랑 같은 질문입니다.
# query = "저는 베트남에서 중학교에 간후 졸업했는데, 이럴때 고용허가제를 통해 직업을 얻을 수 있나요?" # 위의 질문을 변형함
# query = "사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?" # E9 Visa Guide 를 위한 질문

response = query_engine.query(query)
response.print_response_stream()

네, 중학교 졸업증만 가지고 있어도 한국에서 일할 수 있습니다. 비전문취업(E-9) 사증은 특정 학력을 요구하지 않으며, 외국인근로자의 고용 등에 관한 법률의 규정에 의한 국내 취업요건을 갖춘 자가 해당됩니다. 다만, E-9 사증을 받기 위해서는 사증발급인정서를 받아야 하며, 범죄경력증명서 및 건강상태확인서를 제출해야 합니다. 또한, 고용허가제에 따라 적절한 고용주를 찾고, 해당 고용주가 외국인 근로자를 고용할 수 있는 조건을 충족해야 합니다.

In [61]:
# query = "중학교 졸업증만 있는데 한국에서 일할 수 있어요?" # E9 Visa FAQ 를 위한 질문, FAQ에 있는 질문 내용이랑 같은 질문입니다.
query = "저는 베트남에서 중학교에 간후 졸업했는데, 이럴때 고용허가제를 통해 직업을 얻을 수 있나요?" # 위의 질문을 변형함
# query = "사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?" # E9 Visa Guide 를 위한 질문

response = query_engine.query(query)
response.print_response_stream()

네, 중학교 졸업증만 가지고 있어도 한국에서 일할 수 있습니다. 비전문취업(E-9) 사증은 특정 학력을 요구하지 않으며, 외국인근로자의 고용 등에 관한 법률의 규정에 의한 국내 취업요건을 갖춘 자가 해당됩니다. E-9 사증을 받기 위해서는 사증발급인정서를 받아야 하며, 범죄경력증명서 및 건강상태확인서를 제출해야 합니다. 고용허가제에 따라 적절한 고용주를 찾고, 해당 고용주가 외국인 근로자를 고용할 수 있는 조건을 충족해야 합니다.

In [62]:
# query = "중학교 졸업증만 있는데 한국에서 일할 수 있어요?" # E9 Visa FAQ 를 위한 질문, FAQ에 있는 질문 내용이랑 같은 질문입니다.
# query = "저는 베트남에서 중학교에 간후 졸업했는데, 이럴때 고용허가제를 통해 직업을 얻을 수 있나요?" # 위의 질문을 변형함
query = "사증발급인정서를 받기 위해 제출해야 하는 서류에는 어떤 것들이 있나요?" # E9 Visa Guide 를 위한 질문

response = query_engine.query(query)
response.print_response_stream()

사증발급인정서를 받기 위해 제출해야 하는 서류는 다음과 같습니다:

공통서류:
- 사증발급인정신청서(별지제21호서식)
- 여권
- 표준규격사진 1매
- 사업자등록증 사본
- 고용허가서 및 표준근로계약서 사본
- 사업장실태조사서

업종별 추가서류:
- 제조업: 추가서류 없음
- 건설업: 건설업등록증, 도급(하도급) 계약서, 건설현장에 대한 근무 중 외국인력 현황표(해당 현장 책임 건설업〈원도급업체〉 작성)
- 농축산업: 농업경영체등록(변경등록)확인서(국립농산물품질관리원장 발급, 유효기간 1년)
- 어업: 연근해어업의 경우 선박(어선)검사증서, 어업허가증(연안어업, 근해어업, 구획어업), 또는 어업면허증 및 관리선사용지정증(정치망어업), 양식어업의 경우 선박(어선)검사증서, 어업허가증 또는 어업면허증 또는 내수면어업신고필증, 천일염생산 및 암염채취업의 경우 염제조업허가증 사본, 염전임차인의 경우 시장․군수가 발급하는 염제조업 확인서와 임대차계약서, 임대인의 염제조허가증
- 서비스업: 추가서류 없음

또한, 고용허가제 해당자는 범죄경력증명서 및 건강상태확인서를 제출해야 합니다.